In [1]:
import numpy as np
from sparse_beam import sparse_beam, sim_sparse_beam
from pyuvdata import UVBeam
import time
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cProfile

In [2]:
def testing(fname, freq, nsource, profile_it=False):
    def frange(f):
        if freq is None:
            return "None"
        elif len(f) == 1:
            return str(f[0])
        else:
            return str(f[0])+"..."+str(f[-1])

    
    
    uvb = UVBeam()
    uvb.read_beamfits(fname)
    uvb.interpolation_function = "az_za_simple"
    uvb.freq_interp_kind = "linear"


    print("Beam file", fname)

    repeats = 5

    if freq == "all_explicit":
        freq = uvb.freq_array[0]

    elif freq is not None and len(freq) == 1:
        uvb_one_freq = uvb.interp(freq_array=freq, new_object=True, run_check=False)
 
    if nsource is None:
        az = None
        za = None
    else:
        az = np.random.random(size=nsource)*2*np.pi
        za = np.random.random(size=nsource)*np.pi/2

    print("-"*50)
    print("uvbeam, interp freq: "+frange(freq)+",", "nsource: "+str(nsource))
    for i in range(repeats):
        if profile_it:
            cProfile.runctx("uvb.interp(az_array=az, za_array=za, freq_array=freq)", globals(), locals())
        else:
            start = time.time()
            result = uvb.interp(az_array=az, za_array=za, freq_array=freq)
            print("Time:", time.time()-start, "s" if i == 0 else "s, (repeated)")

    if freq is not None and len(freq) == 1:
        print("-"*50)
        print("uvbeam stripped to interp freq, interp freq: "+frange(freq)+",", "nsource: "+str(nsource))
        for i in range(repeats):
            if profile_it:
                cProfile.runctx("uvb_one_freq.interp(az_array=az, za_array=za, freq_array=freq)", globals(), locals())
            else:
                start = time.time()
                uvb_one_freq.interp(az_array=az, za_array=za, freq_array=freq)
                print("Time:", time.time()-start, "s" if i == 0 else "s, (repeated)")

    print("-"*50)
    print("uvbeam optimized, interp freq: "+frange(freq)+",", "nsource: "+str(nsource))
    for i in range(repeats):
        if profile_it:
            cProfile.runctx("uvb.interp(az_array=az, za_array=za, freq_array=freq, reuse_spline=True, check_azza_domain=False)", globals(), locals())
        else:
            start = time.time()
            uvb.interp(az_array=az, za_array=za, freq_array=freq, reuse_spline=True, check_azza_domain=False)
            print("Time:", time.time()-start, "s" if i == 0 else "s, (repeated)")

    if freq is not None and len(freq) == 1:
        print("-"*50)
        print("uvbeam optimized, stripped to interp freq, interp freq: "+frange(freq)+",", "nsource: "+str(nsource))
        for i in range(repeats):
            if profile_it:
                cProfile.runctx("uvb_one_freq.interp(az_array=az, za_array=za, freq_array=freq, reuse_spline=True, check_azza_domain=False)", globals(), locals())
            else:
                start = time.time()
                uvb_one_freq.interp(az_array=az, za_array=za, freq_array=freq, reuse_spline=True, check_azza_domain=False)
                print("Time:", time.time()-start, "s" if i == 0 else "s, (repeated)")

    if uvb.beam_type == "power" and uvb.Nfeeds is None:
        if uvb.future_array_shapes:
            Nfeeds = uvb.data_array.shape[1]
        else:
            Nfeeds = uvb.data_array.shape[2]
    else:
        Nfeeds = None
            

    print("-"*50)
    print("sparse beam, interp freq: "+frange(freq)+",", "nsource: "+str(nsource))
    sb = sparse_beam(fname, nmax=80, mmodes=np.arange(-45, 46), Nfeeds=Nfeeds)    
    for i in range(repeats):
        if profile_it:
            cProfile.runctx("sb.interp(az_array=az, za_array=za, freq_array=freq)", globals(), locals())
        else:
            start = time.time()
            sb_result = sb.interp(az_array=az, za_array=za, freq_array=freq)
            print("Time:", time.time()-start, "s" if i == 0 else "s, (repeated)")

    if freq is not None and len(freq) == 1:
        print("-"*50)
        print("sparse beam, stripped to interp freq, interp freq: "+frange(freq)+",", "nsource: "+str(nsource))   
        sb_one_freq = sb.interp(az_array=None, za_array=None, freq_array=freq)
        for i in range(repeats):
            if profile_it:
                cProfile.runctx("sb_one_freq.interp(az_array=az, za_array=za, freq_array=freq)", globals(), locals())
            else:
                start = time.time()
                sb_result_one_freq = sb_one_freq.interp(az_array=az, za_array=za, freq_array=freq)
                print("Time:", time.time()-start, "s" if i == 0 else "s, (repeated)")
    

    print("-"*50)
    print("sparse beam, sparse fit, interp freq: "+frange(freq)+",", "nsource: "+str(nsource))
    if isinstance(freq, np.ndarray):
        print("\tCan't do it when passing explicit frequencies (this may have changed)")
    else:
        sb = sparse_beam(fname, nmax=80, mmodes=np.arange(-45, 46), Nfeeds=Nfeeds)    
        for i in range(repeats):
            if profile_it:
                cProfile.runctx("sb.interp(az_array=az, za_array=za, freq_array=freq, Nfeeds=Nfeeds)", globals(), locals())
            else:
                start = time.time()
                fit_coeffs, fit_beam = sb.get_comp_fits()
                sb_result = sb.interp(az_array=az, za_array=za, freq_array=freq, sparse_fit=True, fit_coeffs=fit_coeffs)
                print("Time:", time.time()-start, "s" if i == 0 else "s, (repeated)")

    print()
    print("% diff between spaprse and sparse stripped", np.max((sb_result[0]-sb_result_one_freq[0])/result[0]))
    
    if result[0].shape == sb_result[0].shape:
        print("shapes match", result[0].shape, sb_result[0].shape)
    if result[0].dtype == sb_result[0].dtype:
        print("types match", result[0].dtype, sb_result[0].dtype)

In [14]:
az = np.random.random(size=20)*2*np.pi
za = np.random.random(size=20)*np.pi
sb = sparse_beam("NF_HERA_Vivaldi_power_beam.fits", 10, np.arange(10, dtype=int))
x = sb.interp(az_array=az, za_array=za, freq_array=np.array([60e6, 70e6]))[0]
print(x.shape)

(1, 1, 2, 2, 20)


In [4]:
sb = sim_sparse_beam("NF_HERA_Vivaldi_power_beam.fits", 10, np.arange(10, dtype=int))
print(sb.data_array.shape)
sbb = sb.interp(az_array=None, za_array=None, freq_array=np.array([60e6, 61e6]))
print(sbb.data_array.shape, sbb.dA.shape)

(1, 1, 2, 201, 91, 360)
(1, 1, 2, 201, 91, 360) (91, 360)


# Testing interp() of sparse beam

There are three types of measures for testing:

1. It works.
2. The run time compared to the UVBeam version.
3. The values compared to the UVBeam version.
4. The shape and type of the result is the same as UVBeam.

UVBeam can be run in several configurations:

1. With or without without options that make it run faster. The options turn off checking of the input az/za and specify to reuse splines previously generated.
2. IF the interpolation frequencies are just one frequency, a UVBeam can be generated containing just that frequency, which is then used for the interpolation.
3. Combinations of the above.

The test function will run both 1 and 2 if it can. If 2 does not apply, there will be 2 configurations used. If 2 does apply, there will be 4 configurations used.

Frequency 

## Test standard usage

Use all the frequencies in the beam, and 10000 sources.

### efield beam

In [5]:
#testing("NF_HERA_Vivaldi_efield_beam.fits", None, 10000)

### power beam

In [6]:
#testing("NF_HERA_Vivaldi_power_beam.fits", None, 10000)

In [7]:
testing("NF_HERA_Vivaldi_power_beam.fits", np.array([60e6]), 3000)

Beam file NF_HERA_Vivaldi_power_beam.fits
--------------------------------------------------
uvbeam, interp freq: 60000000.0, nsource: 3000
Time: 0.7479126453399658 s
Time: 0.7428886890411377 s, (repeated)
Time: 0.7203216552734375 s, (repeated)
Time: 0.7182595729827881 s, (repeated)
Time: 0.7316915988922119 s, (repeated)
--------------------------------------------------
uvbeam stripped to interp freq, interp freq: 60000000.0, nsource: 3000
Time: 0.7307167053222656 s
Time: 0.7177548408508301 s, (repeated)
Time: 0.7233314514160156 s, (repeated)
Time: 0.7232189178466797 s, (repeated)
Time: 0.7244837284088135 s, (repeated)
--------------------------------------------------
uvbeam optimized, interp freq: 60000000.0, nsource: 3000
Time: 0.010962247848510742 s
Time: 0.0029137134552001953 s, (repeated)
Time: 0.0026733875274658203 s, (repeated)
Time: 0.0026772022247314453 s, (repeated)
Time: 0.002651691436767578 s, (repeated)
--------------------------------------------------
uvbeam optimized,

In [8]:
assert False

AssertionError: 

In [ ]:
sb = sparse_beam("NF_HERA_Vivaldi_power_beam.fits", 10, np.arange(10, dtype=int))
vals = sb.interp(az_array=np.array([0]), za_array=np.array([0]), freq_array=[6e7, 7e7, 8.8e7])[0]
sb.interp(freq_array=np.array([0]), new_object=True, run_check=False)
print(vals.shape)

In [ ]:

az = np.random.random(size=20000)*2*np.pi
za = np.random.random(size=20000)*np.pi

vals = sb.interp(az_array=az, za_array=za, freq_array=[60e6])[0]
print(vals.shape)     # (Naxes_vec, 1, Npols, Nfreqs, Npos)
assert False

### Try using sparse_fit

It expects Nfeeds to be specified but it may not be for a power beam.

In [ ]:
try:
    fit_coeffs, fit_beam = sb.get_comp_fits()
    vals = sb.interp(sparse_fit=True, fit_coeffs=fit_coeffs, az_array=az, za_array=za)[0]
    print(vals.shape)
except Exception as exc:
    import traceback
    print(traceback.format_exc())
    print(exc)

I added some code to sparse_beam so that I can pass Nfeeds and set it in the beam object.

In [ ]:
sb = sparse_beam("NF_HERA_Vivaldi_power_beam.fits", 10, np.arange(10, dtype=int), Nfeeds=2, freq_range=(5e7, 5e7))

In [ ]:
fit_coeffs, fit_beam = sb.get_comp_fits()
vals = sb.interp(sparse_fit=True, fit_coeffs=fit_coeffs, az_array=az, za_array=za)[0]
print(vals.shape)

### Try efield beam

In [ ]:
sb = sparse_beam("NF_HERA_Vivaldi_efield_beam.fits", 10, np.arange(10, dtype=int), freq_range=(5e7, 5e7))

In [ ]:
az = np.random.random(size=20000)*2*np.pi
za = np.random.random(size=20000)*np.pi

vals = sb.interp(az_array=az, za_array=za)[0]
print(vals.shape)     # (Naxes_vec, 1, Npols, Nfreqs, Npos)

In [ ]:
fit_coeffs, fit_beam = sb.get_comp_fits()
vals = sb.interp(sparse_fit=True, fit_coeffs=fit_coeffs, az_array=az, za_array=za)[0]
print(vals.shape)

## Compare beam values

In [ ]:
az = np.random.random(size=2000)*2*np.pi
za = np.random.random(size=2000)*np.pi/2

In [ ]:
import cProfile

uvb = UVBeam()
uvb.read_beamfits("NF_HERA_Vivaldi_power_beam.fits")
uvb.interpolation_function = "az_za_simple"
uvb.freq_interp_kind = "linear"

uvb = uvb.interp(freq_array=np.array([60000000.0]), new_object=True, run_check=False)
start = time.time()
uvb_vals = uvb.interp(az_array=az, za_array=za, freq_array=np.array([6e7]), reuse_spline=True, check_azza_domain=False, spline_opts=None)[0]
print("T", time.time()-start)
print(uvb_vals.shape)



sb = sparse_beam("NF_HERA_Vivaldi_power_beam.fits", 80, np.arange(-45, 46, dtype=int), Nfeeds=2)
sb = sb.interp(freq_array=np.array([60000000.0]), new_object=True, run_check=False)

sb_vals = sb.interp(az_array=az, za_array=za, freq_array=[6e7])[0]
cProfile.run("sb.interp(az_array=az, za_array=za, freq_array=[6e7])[0]")
print(sb_vals.shape)


#fit_coeffs, fit_beam = sb.get_comp_fits()
#sb_sparse_vals = sb.interp(sparse_fit=True, fit_coeffs=fit_coeffs, az_array=az, za_array=za)[0]

In [ ]:
plt.figure(figsize=(18, 9))
plt.subplot(3, 1, 1)
plt.title("Interpolated beam values for 1000 source locations\n")
plt.plot(uvb_vals[0,0,0,0], lw=0.5, color="blue", label="uvbeam")
plt.legend()
plt.subplot(3, 1, 2)
plt.plot(np.abs(sb_vals[0,0,0,0]), lw=0.5, color="teal", label="sparse_beam")
plt.legend()
plt.subplot(3, 1, 3)
plt.plot(np.abs(sb_sparse_vals[0,0,0,0]), lw=0.5, color="orange", label="sparse_beam+sparse fit")
plt.legend()


In [ ]:
plt.figure(figsize=(18, 9))
plt.subplot(3, 1, 1)
plt.title("Differences of interpolated beam values for 1000 source locations\n")
plt.plot(uvb_vals[0,0,0,0]-np.abs(sb_vals[0,0,0,0]), lw=0.5, color="blue", label="difference(uvbeam, sparse_beam)")
plt.legend()
plt.subplot(3, 1, 2)
plt.plot(uvb_vals[0,0,0,0]-np.abs(sb_sparse_vals[0,0,0,0]), lw=0.5, color="teal", label="difference(uvbeam, sparse_beam+sparse fit)")
plt.legend()
plt.subplot(3, 1, 3)
plt.plot(np.abs(sb_vals[0,0,0,0])-np.abs(sb_sparse_vals[0,0,0,0]), lw=0.5, color="orange", label="difference(sparse_beam, sparse_beam+sparse fit)")
plt.legend()

In [ ]:
class Grid:
    def __init__(self, dim):
        self.grid = np.zeros((dim, dim))  
        self.dim = dim
        self.num = np.zeros((dim, dim), dtype=int)
        
    def insert(self, x, y, val):   # the physical values range from -1 to 1
        i = np.trunc(np.interp(x, [-1, 1], [0, self.dim-1])).astype(int)
        j = np.trunc(np.interp(y, [-1, 1], [0, self.dim-1])).astype(int)
    
        for k in range(val.size):
            self.grid[i[k], j[k]] += val[k]
            self.num[i[k], j[k]] += 1
        
        """
        for i in range(self.dim):
            for j in range(self.dim):
                if self.grid[i, j]>0 and self.num[i][j] == 0: print("G")
                if self.grid[i, j]==0 and self.num[i][j] > 0: print("G")
        """      

    def insert1(self, x, y, val):   # the physical values range from -1 to 1
        i = np.trunc(np.interp(x, [-1, 1], [0, self.dim-1])).astype(int)
        j = np.trunc(np.interp(y, [-1, 1], [0, self.dim-1])).astype(int)
        
        self.grid[i, j] += val
        self.num[i, j] += 1
    
            
    def average(self):
        for i in range(self.dim):
            for j in range(self.dim):
                if self.grid[i, j] > 0:
                    self.grid[i, j] /= self.num[i, j]

        #self.grid = np.where(self.grid>0, self.grid/self.num, self.grid)

        
    def log10(self):
        vmin = 1e9
        for i in range(self.dim):
            for j in range(self.dim):
                if self.grid[i, j] > 0:
                    self.grid[i, j] = np.log10(self.grid[i, j])
                    if self.grid[i, j] < vmin: vmin = self.grid[i, j]
                else:
                    self.grid[i, j] = np.nan
                    
        return vmin
        #self.grid = np.where(self.grid>0, np.log10(self.grid), self.grid)
    
    def dB(self):
        vmin = self.log10()
        vmin *= 10
        self.grid *= 10
        return vmin


In [ ]:
np.meshgrid(np.arange(5), np.arange(3))

In [ ]:
uvb = UVBeam()
uvb.read_beamfits("NF_HERA_Vivaldi_power_beam.fits", freq_range=(6e7, 6e7))
uvb.interpolation_function = "az_za_simple"
uvb.freq_interp_kind = "linear"

vals = uvb.interp(az_array=az, za_array=za)[0]
print(vals.shape)
interp_beam = vals[0, 0, 0, 0]       
print(interp_beam[:10])


grid = Grid(100)
r = np.sin(za)
x = r*np.sin(az)
y = r*np.cos(az)

grid.insert(-x, y, values)
grid.average()
grid.grid = np.where(grid.grid==0, np.nan, grid.grid)


plt.figure(figsize=(8, 8))
ax = plt.axes()
ax.axis("off")
im=plt.imshow(grid.grid, interpolation="quadric", norm=LogNorm(), cmap="rainbow")
plt.colorbar(im,fraction=0.046, pad=0.04)
plt.title("Vivaldi power beam\n")
plt.savefig("vivaldi_beam")

In [ ]:
_az = np.deg2rad(np.arange(361))
_za = np.deg2rad(np.arange(91))
az = np.tile(_az, _za.size)
za = np.repeat(_za, _az.size)
sb = sparse_beam("NF_HERA_Vivaldi_power_beam.fits", 80, np.arange(-45,46) , Nfeeds=2, freq_range=(6e7, 6e7))
vals = sb.interp(az_array=az, za_array=za)[0]
print(vals.shape)     # (Naxes_vec, 1, Npols, Nfreqs, Npos)

interp_beam = vals[0, 0, 0, 0]       
print(interp_beam[:10])
values = np.real(interp_beam)


grid = Grid(100)
r = np.sin(za)
x = r*np.sin(az)
y = r*np.cos(az)

grid.insert(-x, y, values)
grid.average()
grid.grid = np.where(grid.grid==0, np.nan, grid.grid)


plt.figure(figsize=(8, 8))
ax = plt.axes()
ax.axis("off")
im=plt.imshow(grid.grid, interpolation="quadric", norm=LogNorm(), cmap="rainbow")
plt.colorbar(im,fraction=0.046, pad=0.04)
plt.title("sparse beam (Vivaldi power beam)\n")
plt.savefig("sparse_beam")

In [ ]:
sb = sparse_beam("NF_HERA_Vivaldi_power_beam.fits", 80, np.arange(-45,46) , Nfeeds=2, freq_range=(6e7, 6e7))
fit_coeffs, fit_beam = sb.get_comp_fits()
vals = sb.interp(sparse_fit=True, fit_coeffs=fit_coeffs, az_array=az, za_array=za)[0]
print(vals.shape)     # (Naxes_vec, 1, Npols, Nfreqs, Npos)

interp_beam = vals[0, 0, 0, 0]       
print(interp_beam[:10])
values = np.real(interp_beam)


grid = Grid(100)
r = np.sin(za)
x = r*np.sin(az)
y = r*np.cos(az)

grid.insert(-x, y, values)
grid.average()
grid.grid = np.where(grid.grid==0, np.nan, grid.grid)


plt.figure(figsize=(8, 8))
ax = plt.axes()
ax.axis("off")
im=plt.imshow(grid.grid, interpolation="quadric", norm=LogNorm(), cmap="rainbow")
plt.colorbar(im,fraction=0.046, pad=0.04)
plt.title("sparse beam+sparse fit (Vivaldi power beam)\n")
plt.savefig("sparse_beam+sparse_fit")